In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time
import re
import progressbar
from dateutil.parser import parse
import datetime

In [ ]:
import twint
import nest_asyncio

d_begin = "2022-03-10"
d_end = "2022-04-02"
daily_datarange = pd.date_range(start=d_begin,
                                end=d_end ,
                                freq='b')

nest_asyncio.apply()
tweet_ids = []
tweet_urls  = []

for start_i in range(0, len(daily_datarange), 1):
    c = twint.Config()
    c.Username = "cnn"      
    c.Pandas = True
    c.Since = str(daily_datarange[start_i].strftime("%Y-%m-%d"))
    c.Until = str((daily_datarange[start_i] + datetime.timedelta(days=2)).strftime("%Y-%m-%d"))
    c.Store_csv = True
    c.Custom_csv = ["id", "user_id", "username", "tweet"]
    c.Output = ("C:/Users/13a71/Dropbox/github/crawler/tweeter crawler/twint/cnn/{}.csv".format(c.Since))
    c.Filter_retweets = True
    # Run
    twint.run.Search(c)
    
    
    tweet_urls.extend([str(tweet.urls) for tweet in tweet_ids])


# load data

In [ ]:
searchlist = pd.read_excel('~/documents/cnn_raw_tweet0402.xlsx').urls

In [ ]:
#Open/reset progressbar and dataframes
# pbar = progressbar()

#Create dataframe for storing article contents
content_df = pd.DataFrame(columns=['date', 'title', 'category', 'author', 'content', 'url'])

#Create dataframe for errorlist
error_df = pd.DataFrame(columns=['url'])

In [ ]:
import re
from dateutil.parser import parse
from bs4 import NavigableString

def get_soup(url, headers=None):
    """
    Arguments
    ---------
    url : str
        Web page url
    headers : dict
        Headers for requests. If None, use Mozilla/5.0 as default user-agent

    Returns
    -------
    soup : bs4.BeautifulSoup
        Soup format web page
    """

    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    r = requests.get(url, headers=headers)
    html = r.text
    page = BeautifulSoup(html, 'lxml')
    return page

doublespace_pattern = re.compile('\s+')
lineseparator_pattern = re.compile('\n+')

def normalize_text(text):
    text = text.replace('\t', ' ')
    text = text.replace('\r', ' ')
    text = lineseparator_pattern.sub('\n', text)
    text = doublespace_pattern.sub(' ', text)
    return text.strip()

def to_string(instance):
    final_string = ''
    if isinstance(instance, NavigableString):
        return instance
    for contents in instance.contents:
        final_string += to_string(contents)
    return final_string


In [ ]:
## CNN SCRAPER
def parse_article(url):
    try:
        soup = get_soup(url)
    except:
        print ('trying again in 10 seconds')
        time.sleep(0.1)
        soup = get_soup(url)
    #Get Kicker (catergory)
    try:
        links_all= soup.find_all('link')
        links = [url['href'] for url in links_all if "https://www.cnn.com/" in url['href']]
        main_url = links[0]
        kicker = main_url.split("/")[6]
    except:
        kicker = ""

    #Get the title
    try:
        title = soup.find('h1', class_='pg-headline').text
    except:
        print('title error')
        
    #Get date and strip
    try:
        date = '-'.join([str(elem) for elem in (main_url.split("/")[3:6])]) 
    except:
        date = ''

    #Get author
    try:
        author = soup.find('span', class_ = 'metadata__byline__author').text
    except:
        author = ''

    #Get body text and strip advertisement
    try:
        content = soup.find('div', class_ = 'pg-rail-tall__body').text.replace(",", "")
    except:
        content = ""
    #print (body)
    
    json_object = {
          'date': date,
          'title': title,
          'subtitle': '',
          'content': content,
        'author' :author,
          'category': kicker,
          'source': 'CNN',
          'url': url
      }
    return json_object


In [ ]:
def save(json_obj, directory):
    date = json_obj.get('date', '')
    title = json_obj.get('title', '')
    filepath = '{}/{}_{}.json'.format(directory, date, re.sub('[^a-zA-Z ]+',"", title[:50]))
    with open(filepath, 'w', encoding='utf-8') as fp:
        json.dump(json_obj, fp, indent=2, ensure_ascii=False)
        print('scraped {}'.format(json_obj['title'][:10]))
        
## CNN SCRAPER
directory = 'C:/Users/13a71/Documents/crawling output/cnn'
    
import json
for url in searchlist:
    url = url[2:-2]
    try:
        article = parse_article(url)
        save(article, directory)
    except:
        print("this url error")
        
        

In [ ]:
content_df

In [ ]:

content_df.dropna(subset = ["date"], inplace=True)
content_df = content_df.drop_duplicates(subset='title')

In [ ]:
content_df.to_excel('cnn_df.xlsx')

In [ ]:
all_df = pd.concat([all_df, content_df])

In [ ]:

content_df.to_excel('cnn.xlsx')

error_df.to_csv('wp_peace2.csv')

In [ ]:
## dnjtldxjs vhtmxm

In [ ]:
for url in pbar(urls):
    url = url.split('?')[0] + '?outputType=amp'
    try:
        #Check if the url is a WP page and make some unscrapable exceptions
        if "washingtonpost" not in url:
            error_df.loc[len(content_df)] = [url]
            continue
        if "/graphics/" in url:
            error_df.loc[len(content_df)] = [url]
            continue          
  
            
        try:
            r = requests.get(url, headers=headers)
        except:
            try:
                print ('trying again in 10 seconds')
                time.sleep(10)
                r = requests.get(url, headers=headers)
            except:
                print (str(url) + ' TIME OUT could not be scraped')
                continue
        
        
        #Get the soup
        soup = BeautifulSoup(r.text, 'lxml')

        #Get Kicker (catergory)
        try:
            kicker = soup.find('div', 'kicker').text
        except:
            try:
                kicker = soup.find('div', 'headline-kicker').text
            except:
                rint('No Kicker' + url)
                kicker = ""

        #Get the title
        title = soup.find('h1').text
        #print(title)

        #Try to get the subtitle
        try:
            subtitle = soup.find('h2').text
            if "\n" in subtitle:
                subtitle = ""
        except:
            #print('No subtitle'  + url)
            subtitle = ""

        #Get date and strip
        try:
            raw_date = soup.find('div', 'display-date').text
            date = datetime.strptime(raw_date, "%B %d, %Y at %I:%M %p EDT")
        except:
            try:
                raw_date = soup.find('span', 'author-timestamp')['content'][:10]
                date = datetime.strptime(raw_date, "%Y-%M-%d")
            except:
                try:
                    raw_date = soup.find('div', 'date')['content'][:16]
                    date = datetime.strptime(raw_date, "%Y-%M-%d %H:%M")
                except:
                    try:
                        raw_date = soup.find('div', 'display-date').text
                        date = datetime.strptime(raw_date, "%B %d, %Y at %I:%M %p EST")
                    except:
                        print('No date'  + url)
                        date = ""

        #Get author
        try:
            author = soup.find('span', 'author-name').text.replace(",", "")
        except:
            author = soup.find('a', 'author-name').text.replace(",", "")

        #Get body text and strip advertisement
        try:
            full_body = soup.find('div', 'article-body')
            filtered_body = ""
            for i in full_body.find_all('p', class_='font-copy'): 
                #if (not 'hide-for-print' in str(i) and not 'Read more:' in str(i)):
                if not 'hide-for-print' in str(i):
                    filtered_body += " " + i.text
            body = filtered_body.split('Read more')[0]
        except:
            full_body = soup.find('article', 'pg-article')
            body = ""
            for i in full_body.find_all('p', class_='pg-body-copy'): 
                body += " " + i.text
        #print (body)
        
        content_df.loc[len(content_df)] = [date, title, subtitle, kicker, author, body.replace(",", ""), url]
    
    except:
        error_df.loc[len(content_df)] = [url]
        continue

In [ ]:
content_df

In [ ]:

content_df = content_df.loc[content_df["date"] >= "2021-07-24"].reset_index(drop=True)
content_df = content_df.loc[content_df["date"] <= "2021-08-04"].reset_index(drop=True)